## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-06-28-09 +0000,startribune,"Takeaways: Wild give up three leads, lose to I...",https://www.startribune.com/takeaways-wild-giv...
1,2026-01-11-05-44-56 +0000,bbc,Iran protesters defy crackdown as videos show ...,https://www.bbc.com/news/articles/cm2jek15m8no...
2,2026-01-11-05-02-55 +0000,wapo,Trump officials consider military options as I...,https://www.washingtonpost.com/world/2026/01/1...
3,2026-01-11-05-01-08 +0000,nyt,India’s Ties With Bangladesh Fray as Elections...,https://www.nytimes.com/2026/01/11/world/asia/...
4,2026-01-11-05-00-21 +0000,nypost,Tennessee mom loses leg in attack by her own p...,https://nypost.com/2026/01/11/us-news/tennesse...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2475/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,27
55,ice,20
145,venezuela,18
12,iran,16
60,minneapolis,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
59,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...,82
121,2026-01-10-15-19-51 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,67
81,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,66
6,2026-01-11-04-16-31 +0000,nyt,Anti-ICE Protests Spread Nationwide After Minn...,https://www.nytimes.com/2026/01/10/us/ice-shoo...,65
134,2026-01-10-13-58-00 +0000,wsj,Trump Presses Oil Executives to Invest in Vene...,https://www.wsj.com/business/energy-oil/trump-...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
59,82,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...
121,67,2026-01-10-15-19-51 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...
130,49,2026-01-10-14-18-09 +0000,nypost,Iran activates ‘missile cities’ as protests ra...,https://nypost.com/2026/01/10/world-news/iran-...
73,34,2026-01-10-20-53-00 +0000,wsj,The Defense Department has selected Owen West ...,https://www.wsj.com/politics/national-security...
89,30,2026-01-10-18-52-27 +0000,nypost,"Ilhan Omar squares off against feds, joins Min...",https://nypost.com/2026/01/10/us-news/ilhan-om...
100,30,2026-01-10-17-47-13 +0000,nypost,New NYC Council GOP leader ready to battle Mam...,https://nypost.com/2026/01/10/us-news/new-nyc-...
140,29,2026-01-10-13-00-00 +0000,latimes,One driver accused of two crashes that left th...,https://www.latimes.com/california/story/2026-...
58,28,2026-01-10-22-10-13 +0000,nypost,Thousands pack lefty activist group Monarca’s ...,https://nypost.com/2026/01/10/us-news/minnesot...
125,25,2026-01-10-14-44-49 +0000,cbc,Ukraine scrambles to repair ruined Kyiv power ...,https://www.cbc.ca/news/world/ukraine-russia-p...
104,24,2026-01-10-17-23-27 +0000,nypost,US used powerful mystery weapon that brought V...,https://nypost.com/2026/01/10/world-news/us-us...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
